## Pandas for data analysis

*"Pandas is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language."*

It is a popular library for data analysis of "big data". Pandas offer an easy-to-use interface to quickly load and manipulate data in some of the most common formats (see http://pandas.pydata.org/pandas-docs/version/0.20/io.html for details).

For tips and tricks on how to use Pandas for analysis of really big data (> 100MB, up to several GBs) check https://www.dataquest.io/blog/pandas-big-data/ . This page also offers information on internals of the Pandas library.

More information about libraries:
Pandas http://pandas.pydata.org/
Numpy http://www.numpy.org/


In [ ]:
#IMPORTS - if you don't run Anaconda, you will need to install Pandas and Numpy module first (use PyPi)
# pip install pandas

import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

Dataset used from http://www.mapakriminality.cz/data contains information on criminal activities in CR between years 2013-2017

In [ ]:
#Variables
ROW_NAMES = ["Index Kriminality","Zjištěno","Objasněno- Počet","Objasněno- Dodatečně", \
             "Stíháno, Vyšetřováno Osob - Mladiství 15-17 Let","Stíháno, Vyšetřováno Osob - Recidivisté", \
             "Stíháno, Vyšetřováno Osob - Celkem","Stíháno, Vyšetřováno Osob - Nezletilí 1-14 Let", \
             "Stíháno, Vyšetřováno Osob - Ženy","Spácháno Skutků - Mladiství 15-17 Let", \
             "Spácháno Skutků - Z Toho Alkohol","Spácháno Skutků - Pod Vlivem",\
             "Spácháno Skutků - Recidivisté","Spácháno Skutků - Nezletilí 1-14 Let",\
             "Škody V Tis. Kč - Zajištěno","Škody V Tis. Kč - Celkem","Časová Jednotka",\
             "Kód úz.jednotky","Název úz.jednotky","Počet Obyv. úz. Jednotky"]
USED_ROW_NAME = 'Zjištěno'
TIMESTAMP_NAME = "Časová Jednotka"

In [ ]:
# Read CSV file
df = pd.read_csv('data//crimes-0100-101-903-120-167.csv')

In [ ]:
# Show loaded data, use df.head(n = 5) for larger datasets
df.head()

In [ ]:
# Show names of columns
df.columns

In [ ]:
# Numpy arrays work the same way as Python lists

# Only show data in column we want to work with (head is to show only the beginning of the array, often used for performance)
df[USED_ROW_NAME].head()

# Show specific value
df[USED_ROW_NAME][0]

In [ ]:
# Show types of data we work with
df.dtypes

In [ ]:
# Lambda functions in python is limited to a single command (nevertheless this command can be very complex)
#       - you can pass multiple input parameters into the lambda function

##f = lambda x,y : x+y
##f(1,2)

#       - you can call if/else structure, but "else' branch needs to be present

##f = lambda x : x * x if x > 0 else "IT'S A TRAP!"
##f(5)

#       - you CAN'T use for circle
##f = lambda x,y : x*x for y in range(10)

In [ ]:
# Change all 0 values to NaN
df = df.applymap(lambda x: np.nan if x == 0 else x)
df

In [ ]:
# Show information about our dataset
df[USED_ROW_NAME].describe()

In [ ]:
# Change date format so that year is first (note that data is already sorted, if it wasn't use method groupby)
df[TIMESTAMP_NAME] = df[TIMESTAMP_NAME].map(lambda x: datetime.strptime(x, '%m-%Y').date())
df

In [ ]:
# Index for data frame is set to be the datetime
df.set_index(pd.DatetimeIndex(df[TIMESTAMP_NAME]), inplace=True)

# Sum up all data for USED_ROW_NAME with same month
def sum_up_months(df):
    column = []
    
    for current_month in range(1, 13):
        # Choose all lines with the same month
        all_lines = df[df.index.month == current_month]
        # Sum values in all lines picked
        column.append(all_lines[USED_ROW_NAME].sum())
        
    return column

# Create new data frame, that contains aggregated data
cols = sum_up_months(df)
df_ = pd.DataFrame(index=range(12), columns=[USED_ROW_NAME])
df_[USED_ROW_NAME] = cols

# Check if we got all months (indexing starts with 0)
df_.head()

In [ ]:
# Draw the graph in matplotlib
%matplotlib inline 
x_axis = list(range(len(df[USED_ROW_NAME])))
x_ticks = [str(x) for x in df[TIMESTAMP_NAME]]

#Set size for inline plot
plt.rcParams['figure.figsize'] = (20.0, 10.0)

#Set X axis names and draw graph
plt.xticks(x_axis, x_ticks)
plt.bar(x_axis, df[USED_ROW_NAME], color="orange", width=0.5)


In [ ]:
#Only show first 12 months
plt.xticks(x_axis[:12], x_ticks[:12])
plt.bar(x_axis[:12], df[USED_ROW_NAME][:12], color="orange", width=0.5)

In [ ]:
#Agregated data

x_axis = list(range(len(df_[USED_ROW_NAME])))
x_ticks = [str(x+1) for x in df_.index]

plt.xticks(x_axis, x_ticks)
plt.bar(x_axis, df_[USED_ROW_NAME], color="orange", width=0.5)

In [ ]:
# Save times and used column data to CSV file
df[[TIMESTAMP_NAME,USED_ROW_NAME]].to_csv("cleaned_data.csv", header=True)